<a href="https://colab.research.google.com/github/ajou-y0uC4N/y0uC4N/blob/main/DNN_case10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/type10.csv")
print(df.head())

     type      time  isCan0  priority  messageID  isService  serviceNODEID  \
0  Normal  0.000000       1         5       1030          0              1   
1  Normal  0.000139       1         5       1030          0              1   
2  Normal  0.004482       1         5       1030          0              1   
3  Normal  0.004641       1         5       1030          0              1   
4  Normal  0.009330       1         5       1030          0              1   

   byteSize  data[0]  data[1]  data[2]  data[3]  data[4]  data[5]  data[6]  \
0         8      166       53        0        0        0        0        0   
1         7        0        0        0        0        0        0      116   
2         8      166       53        0        0        0        0        0   
3         7        0        0        0        0        0        0      117   
4         8      166       53        0        0        0        0        0   

   data[7]  7byteNullCnt  8byteNullCnt  
0      148           

In [4]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df["type"])
X = df.drop(columns=["type",'time'])

print(f"X shape: {X.shape}")
print(f"y shape: {y_encoded.shape}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X shape: (207380, 16)
y shape: (207380,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [6]:
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2176      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 12545 (49.00 KB)
Trainable params: 12545 (49.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
4148/4148 [==============================] - 15s 3ms/step - loss: 0.2077 - accuracy: 0.8868 - val_loss: 0.1952 - val_accuracy: 0.8943
Epoch 2/10
4148/4148 [==============================] - 14s 3ms/step - loss: 0.1890 - accuracy: 0.8943 - val_loss: 0.1828 - val_accuracy: 0.8973
Epoch 3/10
4148/4148 [==============================] - 12s 3ms/step - loss: 0.1837 - accuracy: 0.8959 - val_loss: 0.1803 - val_accuracy: 0.9023
Epoch 4/10
4148/4148 [==============================] - 11s 3ms/step - loss: 0.1798 - accuracy: 0.8984 - val_loss: 0.1801 - val_accuracy: 0.9003
Epoch 5/10
4148/4148 [==============================] - 11s 3ms/step - loss: 0.1777 - accuracy: 0.8984 - val_loss: 0.1749 - val_accuracy: 0.9023
Epoch 6/10
4148/4148 [==============================] - 13s 3ms/step - loss: 0.1760 - accuracy: 0.8993 - val_loss: 0.1764 - val_accuracy: 0.9024
Epoch 7/10
4148/4148 [==============================] - 12s 3ms/step - loss: 0.1748 - accuracy: 0.8997 - val_loss: 0.1732 - val_ac

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_,digits=6))

1297/1297 [==============================] - 3s 2ms/step - loss: 0.1726 - accuracy: 0.9001
Test loss: 0.1725778877735138
Test accuracy: 0.9000626802444458
1297/1297 [==============================] - 2s 1ms/step
              precision    recall  f1-score   support

      Attack   0.986490  0.738055  0.844378     15236
      Normal   0.867307  0.994131  0.926399     26240

    accuracy                       0.900063     41476
   macro avg   0.926899  0.866093  0.885388     41476
weighted avg   0.911088  0.900063  0.896269     41476



In [9]:
model.save('my_dnn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
